In [2]:
from maze.LM_Environment import *
from maze.LM_Data import *
from q_learn.LM_Q_Helper import *

import numpy as np
import pandas as pd
import random
import time
from multiprocessing import Pool
import re
import multiprocessing as mp

In [3]:
name_q_table = 'q_table_s2_02'
path = '../data/q_learn/policies/'

q_table = pd.read_csv(f'{path}{name_q_table}_explored.csv')

In [4]:
state = '_                      X _'


idx_q = q_table.state[q_table.state == state].index[0]
idx_q

6

In [30]:
opt_qcols = ['sco_wait', 'sco_up', 'sco_down', 'sco_left', 'sco_right']

states = q_table.state.values
idx_q = np.where(states == state)[0]
pvals = np.array(q_table.loc[idx_q, opt_qcols])
idx_q, pvals

(array([6]),
 array([[0.04926559, 0.0542031 , 0.03228615, 0.0273686 , 0.01426815]]))

In [6]:
q_table.iloc[idx_q, 1:6].values

array([0.0492655875341914, 0.054203097004551, 0.0322861498668558,
       0.0273685987185964, 0.0142681454392827], dtype=object)

In [45]:
policy = q_table.iloc[:, 1:6].values
policy.shape

policy[idx_q][0]

array([0.04926559, 0.0542031 , 0.03228615, 0.0273686 , 0.01426815])

In [112]:
q_table  = pd.read_csv(f'{path}{name_q_table}_explored.csv')
q_table = q_table.iloc[:, 0:6]


id = 16031652
state = '_XXXXXXXXXXXX XXX   XX   _' 
pvals = [0.01892812, 0.01783451, 0.0154507,  0.01783462, 0.02092926]
counts = [  0.,  0.,  0.,  1.,  0.]
score = 2

In [124]:

entry = [state, *[score * c for c in counts]]

print(entry)
new = pd.DataFrame([entry], columns=q_table.columns)
print('new: ',new)

policy = pd.concat((q_table, new), axis=0, ignore_index=True)

print('\nresulting policy:')
policy

['_XXXXXXXXXXXX XXX   XX   _', 0.0, 0.0, 0.0, 2.0, 0.0]
new:                          state  sco_wait  sco_up  sco_down  sco_left  sco_right
0  _XXXXXXXXXXXX XXX   XX   _       0.0     0.0       0.0       2.0        0.0

resulting policy:


,state,sco_wait,sco_up,sco_down,sco_left,sco_right
0,_ _,0.008495,0.006653,0.010567,0.010048,0.012547
1,_ 8_,0.000000,0.000000,0.000000,0.000000,0.000000
2,_ E_,0.000000,0.000000,0.000000,0.000000,0.000000
3,_ X_,0.043021,0.035046,0.043732,0.043858,0.033693
4,_ 8 _,0.069501,0.000000,0.000000,0.084946,0.000000
...,...,...,...,...,...,...
17096064,_X8XXX X XX X X EX_,0.736842,0.736842,0.736842,0.736842,0.736842
17096065,_X8XXX X XXX XX XX XX_,0.000000,0.000000,0.000000,0.000000,0.000000
17096066,_X8XXX X XXX XX XXXX XXX_,0.701354,0.701354,0.701354,0.701354,0.701354
17096067,_X8XXXXX X X X XX X_,0.703581,0.703581,0.703581,0.703581,0.703581


In [14]:
batch_size = 10000
games_data = run_random_games(batch_size, 12, 10, 2, 2, 1, 20, 10)

100%|██████████| 10000/10000 [03:54<00:00, 42.61it/s]


In [26]:
patterns = [  '^_     ', '^_    [8X]',
              '^_   [8X]', 
              '^_  [X8]', 
              '^_ X', '^_X ', 
              '^_XX[ 8]', '^_XXX', 
              '^_8', '^_[ X]8']

print(games_data.shape)
print(q_table.shape)

Qs = []
games = []
shares, hares = 0, 0

for i in range(len(patterns)):
    pat = patterns[i]
    games.append(games_data[games_data.state.str.match(pat)])
    Qs.append(q_table[q_table.state.str.match(pat)])
    
    share = games[i].shape[0]/len(games_data)
    shares += share
    hare = Qs[i].shape[0]/len(q_table)
    hares += hare
    print(f'PAT {pat}:  games-{games[i].shape} ({round(share, 2)})    |   Q-table-{Qs[i].shape}({round(hare, 2)}) ')

print(f'shares (games/Qtab): {shares}   {hares}')

(231702, 7)
(16777191, 12)
PAT ^_     :  games-(38546, 7) (0.17)    |   Q-table-(524288, 12)(0.03) 
PAT ^_    [8X]:  games-(10879, 7) (0.05)    |   Q-table-(524288, 12)(0.03) 
PAT ^_   [8X]:  games-(12550, 7) (0.05)    |   Q-table-(1048576, 12)(0.06) 
PAT ^_  [X8]:  games-(12165, 7) (0.05)    |   Q-table-(2097152, 12)(0.13) 
PAT ^_ X:  games-(12326, 7) (0.05)    |   Q-table-(4194303, 12)(0.25) 
PAT ^_X :  games-(32565, 7) (0.14)    |   Q-table-(4194303, 12)(0.25) 
PAT ^_XX[ 8]:  games-(21738, 7) (0.09)    |   Q-table-(2097151, 12)(0.13) 
PAT ^_XXX:  games-(88701, 7) (0.38)    |   Q-table-(2097130, 12)(0.12) 
PAT ^_8:  games-(1091, 7) (0.0)    |   Q-table-(0, 12)(0.0) 
PAT ^_[ X]8:  games-(1141, 7) (0.0)    |   Q-table-(0, 12)(0.0) 
shares (games/Qtab): 1.0   1.0
